# DATA 601 Homework 2
# Convective Weather Analysis Tool
## Python weather analysis application that detects the presence of frontal boundaries and assesses thunderstorm potential.

### Overview: Convective cloud and storm development is a common occurrence over southwest Florida during the warm season (i.e., April - October). This Python code package provides a surface weather analysis technique that assesses the thermodynamic energy state of the atmosphere near the surface (ground) and identifies dynamic factors that favor or inhibit convective storm (i.e., thunderstorm) development over sub-tropical coastal areas.

### Example of cumulus-cloud filled sky over southwest Florida
<img src= "images/IMG_3210.JPEG">

### Abstract:
Convective cloud and storm development is a common occurrence over southwest Florida during the Northern Hemisphere warm season (i.e., April - October). Cumulus clouds develop in convection currents resulting from heating near the Earth’s surface. Deep, moist convection can also pose a threat to public safety and aviation operations. Convective cloud (i.e., cumulus cloud, see https://cloudatlas.wmo.int/en/cumulus-cu.html) and, in some cases, subsequent convective storm development can be effectively anticipated from observed weather conditions near the ground. Necessary ingredients for convective cloud development typically includes (1) a moist layer of sufficient depth in the lower atmosphere, and (2) adequate lifting of an air parcel from the moist layer to allow free convection (Johns and Doswell 1992). Equivalent potential temperature (theta-e) can indicate the amount of moist heating near the ground and the resultant instability that favors intense updrafts (i.e., vertically moving air at a high velocity). Lifting of an air parcel from the ground surface typically results from wind convergence (confluence) along the frontal boundary, defined as an air mass discontinuity. Theta-e values greater than 330 degrees Kelvin (K) strongly indicate the potential for deep, moist convective storms (i.e., thunderstorms), especially in the presence of a frontal boundary. Over southwestern Florida, sea-breeze fronts and cold fronts are the most commonly occurring. This application will obtain surface weather observations over a roughly 1-degree latitude by 1-degree longitude boxed area and perform a function to infer the presence of wind convergence and an associated frontal boundary. The theta-e parameter will then be analyzed to determine the potential development of cumulus clouds and thunderstorm activity.

### Research Questions:
1. What is the relationship between surface equivalent potential temperature patterns and thunderstorm potential?
2. How does dynamic forcing (i.e., divergence, convergence) interact with thermodynamic instability in thunderstorm initiation?
3. How can this relationship be quantified, modeled, and assessed?

### Methodology:
This notebook generates tables, weather reports, and diagnostic output. Consists of five blocks that are executed in succession:
- Invoke Pandas to read and load weather observation data from [OpenWeather API](https://openweathermap.org/current) in JSON format from stations within the defined rectangular boundary.
- Define and execute a function that calculates equivalent potential temperature (i.e., theta-e) from temperature and relative humidity values.
- Invoke json_normalize function to generate a regional weather report in table format. Extract weather observation data and print weather reports for two selected locations within the rectangular boundary.
- Execute a decision tree to 1) determine the presence of a frontal boundary, and 2) determine the possibility of deep, moist convective storms (thunderstorms).
- Build dictionaries and a data frame containing weather report and derived parameter data; save data frame as a .csv file in the “data” directory.


## Invoke Pandas to read and load weather observation data in JSON format from stations within the defined rectangular boundary.

In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd 
import sys
from pandas.io.json import json_normalize 
import os, ssl

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

data = pd.read_json(r'https://api.openweathermap.org/data/2.5/box/city?bbox=-82,25.5,-81,26.5,10&units=imperial&appid=edc8e66602652591b418f7492de29f82')
print(data)
print("")

#Print weather observation table from JSON dataset
wxobs = json_normalize(data['list']) 
print(wxobs)

   calctime  cnt  cod                                               list
0  0.001692    4  200  {u'clouds': {u'today': 90}, u'name': u'Golden ...
1  0.001692    4  200  {u'clouds': {u'today': 90}, u'name': u'Naples'...
2  0.001692    4  200  {u'clouds': {u'today': 100}, u'name': u'Bonita...
3  0.001692    4  200  {u'clouds': {u'today': 75}, u'name': u'Immokal...

   clouds.today  coord.Lat  coord.Lon          dt       id  main.feels_like  \
0            90      26.19     -81.70  1605103785  4156920            77.47   
1            90      26.14     -81.79  1605103785  4165565            77.41   
2           100      26.34     -81.78  1605103785  4148533            85.23   
3            75      26.42     -81.42  1605103785  4159553            82.11   

   main.grnd_level  main.humidity  main.pressure  main.sea_level  ...  \
0              NaN             88           1008             NaN  ...   
1              NaN             88           1008             NaN  ...   
2           1008.0 

## Define and execute a function that calculates equivalent potential temperature (i.e., theta-e) from temperature and relative humidity values as documented by Andrew Revering, Convective Development, Inc., http://gradsusr.org/pipermail/gradsusr/2010-January/010266.html

In [2]:
def compthetae(temp,RH,pressure):
    tempc = (temp - 32) / 1.8
    print("Temp_C = ", tempc)
    #Calculate theta-e. Units are in degrees Kelvin (K)
    thetae = (273.15 + tempc)*((1000/pressure)**0.286)+(3 * (RH * (3.884266 * 10**
         ((7.5 * tempc)/(237.7 + tempc)))/100))
    return thetae

## Invoke json_normalize function to generate a regional weather report in table format. Extract weather observation data and print weather reports for two selected locations within the rectangular boundary.

In [3]:
#Invoke json_normalize function to generate a regional weather report in table format
wx_data = json_normalize(data.to_dict('series'), 'list') 
print(wx_data)

#Extract weather observation parameters and print weather reports for two selected stations 
print("")
print("Weather Report at Location #1")
loc1 = wx_data['name'][1]
windspd_loc1 = wx_data['wind'][1]['speed']
winddir_loc1 = wx_data['wind'][1]['deg']
temp_loc1 = wx_data['main'][1]['temp']
RH_loc1 = wx_data['main'][1]['humidity']
pressure_loc1 = wx_data['main'][1]['pressure']
thetae_loc1 = compthetae(temp_loc1,RH_loc1,pressure_loc1)
print("Location 1: ", loc1)
print("Wind speed: ", windspd_loc1)
print("Wind direction: ", winddir_loc1)
print("Temperature: ", temp_loc1)
print("Humidity: ", RH_loc1)
print("Pressure: ", pressure_loc1)
print("Theta-e: ", thetae_loc1)

print("")
print("Weather Report at Location #2")
loc2 = wx_data['name'][3]
windspd_loc2 = wx_data['wind'][3]['speed']
winddir_loc2 = wx_data['wind'][3]['deg']
temp_loc2 = wx_data['main'][3]['temp']
RH_loc2 = wx_data['main'][3]['humidity']
pressure_loc2 = wx_data['main'][3]['pressure']
thetae_loc2 = compthetae(temp_loc2,RH_loc2,pressure_loc2)
print("Location 2: ", loc2)
print("Wind speed: ", windspd_loc2)
print("Wind direction: ", winddir_loc2)
print("Temperature: ", temp_loc2)
print("Humidity: ", RH_loc2)
print("Pressure: ", pressure_loc2)
print("Theta-e: ", thetae_loc2)
 

            clouds                            coord          dt       id  \
0   {u'today': 90}   {u'Lat': 26.19, u'Lon': -81.7}  1605103785  4156920   
1   {u'today': 90}  {u'Lat': 26.14, u'Lon': -81.79}  1605103785  4165565   
2  {u'today': 100}  {u'Lat': 26.34, u'Lon': -81.78}  1605103785  4148533   
3   {u'today': 75}  {u'Lat': 26.42, u'Lon': -81.42}  1605103785  4159553   

                                                main            name  \
0  {u'temp': 78.37, u'temp_max': 79, u'humidity':...     Golden Gate   
1  {u'temp': 78.33, u'temp_max': 78.8, u'humidity...          Naples   
2  {u'temp': 77.61, u'grnd_level': 1008, u'temp_m...  Bonita Springs   
3  {u'temp': 78.91, u'temp_max': 79, u'humidity':...       Immokalee   

            rain  snow  visibility  \
0  {u'1h': 3.65}  None        6437   
1  {u'1h': 0.75}  None        6437   
2  {u'1h': 3.65}  None       10000   
3           None  None       10000   

                                             weather  \
0  [{u'main

## Execute a decision tree to 1) determine the presence of a frontal boundary and 2) determine the possibility of deep, moist convective storms (thunderstorms).

In [4]:
#Compute theta-e difference between two stations
delta_thetae = np.absolute(thetae_loc2 - thetae_loc1)
print("Theta-e difference between location #1 and location #2 = ", delta_thetae)

#Test for veering winds between the two stations
if winddir_loc2 < winddir_loc1:
    winddirdiff = np.absolute(winddir_loc1 - winddir_loc2)
    if winddirdiff >= 45: #Test for a significant wind shift between the two stations
        wshift = True
    else:
        wshift = False
    #Test for the presence of a theta-e gradient. If a gradient is present, test for
    #values above two thresholds for severe thunderstorms (>330K) and ordinary thunderstorms (>310K), respectively
    if delta_thetae > 0:
        if thetae_loc2 > 330:
            svrstorms = True
            storms = True
            cumulus = True
        elif thetae_loc2 > 310:
            svrstorms = False
            storms = True
            cumulus = True
        else:
            svrstorms = False
            storms = False
            cumulus = True
    elif thetae_loc2 > 300:
        svrstorms = False
        storms = False
        cumulus = True
    else:
        svrstorms = False
        storms = False
        cumulus = False
        
    div = False #Wind convergence is present
    
    print("")
    
    if wshift == True:
        print("Wind convergence - frontal boundary present")
    else:
        print("Wind convergence - frontal boundary not present")
    if svrstorms == True:
        print("Severe storm and MCS development possible")
    elif storms == True:
        print("Scattered storm development possible")
    else:        
        print("Cumulus cloud development possible")

#Test for backing winds between the two stations        
elif winddir_loc2 > winddir_loc1:
    winddirdiff = np.absolute(winddir_loc1 - winddir_loc2)
    wshift = False
    div = True
    print(wshift,div)
    #Test for the theta-e threshold for cumulus cloud development (310K)
    if thetae_loc2 > 310:
            svrstorms = False
            storms = False
            cumulus = True
    print("Wind divergence - frontal boundary not present")
    if cumulus == True:
        print("Cumulus cloud development possible")    
else:
    winddirdiff = np.absolute(winddir_loc1 - winddir_loc2)
    wshift = False
    div = False
    print(wshift,div)
    if thetae_loc2 > 310:
            svrstorms = False
            storms = False
            cumulus = True
    print("Frontal boundary not present")
    if cumulus == True:
        print("Cumulus cloud development possible")    

Theta-e difference between location #1 and location #2 =  1.3019640595263127
False False
Frontal boundary not present
Cumulus cloud development possible


## Build dictionaries and a data frame containing weather report and derived parameter data; save data frame as a .csv file in the "data" directory.

In [5]:
#Build dictionaries containing weather observation data for two stations
dict_wxdata_loc1 = {'Location 1' : loc1, 'Wind speed' : windspd_loc1, 'Wind direction' : winddir_loc1, 'Temperature' : temp_loc1,
              'Humidity' : RH_loc1, 'Pressure' : pressure_loc1, 'Theta-e:' : thetae_loc1} 

dict_wxdata_loc2 = {'Location 2' : loc2, 'Wind speed' : windspd_loc2, 'Wind direction' : winddir_loc2, 'Temperature' : temp_loc2,
              'Humidity' : RH_loc2, 'Pressure' : pressure_loc2, 'Theta-e:' : thetae_loc2} 
#Build dictionary containing derived parameters
dict_wxanal = {'delta_thetae' : delta_thetae, 'delta_wind_direction' : winddirdiff, 'Wind_shift' : wshift, 'Divergence' : div,
          'Severe_storms' : svrstorms, 'Scattered_storms' : storms, 'Cumulus_clouds' : cumulus} 
print("")
# Directory 
directory = 'data'
#os.mkdir(directory) 
print("Directory '% s' created" % directory) 
print("")
#Build data frame from the dictionaries and save as .csv file in 'data' directory
df_wxanal = pd.DataFrame([dict_wxanal,dict_wxdata_loc1,dict_wxdata_loc2])
df_wxanal = df_wxanal.fillna('')
df_wxanal.to_csv(r'data\wxanal.csv')
print("Weather Analysis Report Dataset")
print(df_wxanal)



Directory 'data' created

Weather Analysis Report Dataset
  Cumulus_clouds Divergence Humidity Location 1 Location 2 Pressure  \
0           True      False                                           
1                                 88     Naples                1008   
2                                 88             Immokalee     1009   

  Scattered_storms Severe_storms Temperature Theta-e: Wind direction  \
0            False         False                                       
1                                      78.33  353.631            120   
2                                      78.91  354.933            120   

  Wind speed Wind_shift delta_thetae delta_wind_direction  
0                 False      1.30196                    0  
1    19.4613                                               
2    12.7505                                               


## Discussion/Summary:
This study demonstrated that equivalent potential temperature (theta-e) can indicate the amount of moist heating near the ground and the resultant instability that favors intense updrafts and convective storm initiation. The presence wind convergence (confluence) or divergence (diffluence) within the study region signified the potential for convective cloud formation. Theta-e values greater than 330 degrees Kelvin (K) strongly indicate the potential for deep, moist convective storms (i.e., thunderstorms), especially in the presence of a frontal boundary. Over southwestern Florida, sea-breeze fronts and cold fronts are the most commonly occurring. This application obtained surface weather observations over a roughly 1-degree latitude by 1-degree longitude boxed area and performed a function to infer the presence of wind convergence and an associated frontal boundary. 

## Further research: 
Validate frontal and storm diagnosis against regional remote sensing datasets, including weather radar and satellite imagery.

## References: 

Johns, R. H., and C. A. Doswell, 1992: Severe local storms forecasting. Wea. Forecasting, 7, 588–612, DOI:10.1175/ 1520-0434(1992)007,0588:SLSF.2.0.CO;2.

Revering, Andrew. “Equivalent Potential Temperature.” gradsusr.org/pipermail/gradsusr/2010-January/010266.html. Accessed 8 October 2020.